<a href="https://colab.research.google.com/github/mrrajatgarg/Music_Classification_And_Tagging_/blob/master/final_2D_CNN_MagnaTagATune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('/content/drive/My Drive')

In [0]:
!ls

'all_images (1).zip'
 all_images.zip
 birds.zip
 catVsdogLogisticRegression.ipynb
 coco.zip
'Colab Notebooks'
 converting_to_final_y_train_val-test.ipynb
 cub.tar.gz
 current_2d_cnn_model.ipynb
 data_clip_id_melgram.zip
 dataset_clip_id_melgram_1.zip
 dataset_clip_id_melgram_2.zip
 dataset_clip_id_melgram_3.zip
 dataset_clip_id_melgram_4.zip
 dataset_clip_id_melgram_5.zip
 dataset_clip_id_melgram.zip
 dataset_clip_id_mp3.zip
 dataset.zip
 Digit_Recognizer_Kaggle.ipynb
 dogsvscatsCNN_final.ipynb
 electrical_classification_model.h5
 electrical_classification_model.ipynb
 filename.csv
'filenames (1).npy'
 filenames.npy
'filenames_shuffled (1).npy'
 filenames_shuffled.npy
 final_Dataframe.pkl
 final_dataset.zip
 final_final_Dataframe.pkl
 Final_Rajat_MagnaTagATune_Dataset_Preparation_1.ipynb
 final_testing_data_51.pkl
 final_training_data_51.pkl
 final_train.npy
 final_validation_data_51.pkl
 final_y_test.pkl
'final_y_train (1).pkl'
 final_y_train.pkl
'final_y_val (1).pkl'
 final_y_val.pkl

# Importing Libraries

In [0]:
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize

import tensorflow as tf
from keras import backend as K

In [0]:
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint

import kapre
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D

In [0]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

# Importing dataset

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/My Drive/dataset_clip_id_melgram.zip', 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [0]:
input_shape = (96, 1366, 1)
print(input_shape)

(96, 1366, 1)


In [0]:
#y_val = np.load('/content/drive/My Drive/final_y_val.pkl')

training_file_path = np.load('/content/drive/My Drive/training_files_names_final_2.npy')
y_train_numpy = np.load('/content/drive/My Drive/training_labels_final_2.npy')

validation_file_path = np.load('/content/drive/My Drive/validation_files_names_final_2.npy')
y_val_numpy = np.load('/content/drive/My Drive/validation_labels_final_2.npy')

testing_file_path = np.load('/content/drive/My Drive/testing_files_names_final_2.npy')
y_test_numpy = np.load('/content/drive/My Drive/testing_labels_final_2.npy')

In [0]:
print(training_file_path.shape)
print(y_train_numpy.shape)

print(validation_file_path.shape)
print(y_val_numpy.shape)

print(testing_file_path.shape)
print(y_test_numpy.shape)

(18706,)
(18706, 50)
(1825,)
(1825, 50)
(5329,)
(5329, 50)


# Testing from things -- delete after testing

In [0]:
training_file_path[0:10]

array([ 3642, 40434, 11746, 28669, 56563, 26289, 55637, 12796, 56111,
       11209])

In [0]:
batch_x_final = np.zeros((0, 96, 1366, 1))
for filename in training_file_path[0:10]:
          full_path = '/content/'+str(filename)+'.npy'
          this_array = np.load(full_path).reshape(1, 96, 1366, 1)
          batch_x_final = np.concatenate((batch_x_final, this_array), axis=0)

In [0]:
print(batch_x_final.shape)

(10, 96, 1366, 1)


In [0]:
print(batch_x_final[0])

# For checkpointing of model

In [0]:
os.chdir('/content/')

!mkdir new_model_saved

In [0]:
#!mkdir saved_models_3

In [0]:
os.chdir('/content/new_model_saved/')

In [0]:
filepath='weights-improvement-4-{epoch:02d}-{val_acc:.2f}.hdf5'

In [0]:
checkpoint = ModelCheckpoint(filepath, verbose=1)

# Designing Model

In [0]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [0]:
model = Sequential()

# Layer 1
model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 4))) 
model.add(Dropout(0.5))

# Layer 2
model.add(Conv2D(384, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 5))) 
model.add(Dropout(0.5))

model.add(Conv2D(768, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,8)))
model.add(Dropout(0.5))
          
model.add(Conv2D(2048, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(4, 8))
model.add(Dropout(0.5))
model.add(Flatten())

#model.add(Dense(100, activation='relu')) 
# model.add(BatchNormalization())
model.add(Dense(50,activation='sigmoid'))

model.compile(loss='binary_crossentropy' , optimizer='adam' , metrics=['accuracy', auc])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 1366, 128)     1280      
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1366, 128)     512       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1366, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 341, 128)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 341, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 341, 384)      442752    
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 341, 384)      1536      
__________

In [0]:
class My_Generator(keras.utils.Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)

    def __getitem__(self, idx):
        #print(self.batch_size)
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        #print(batch_y)
        #batch_y = np.array(batch_y)
        #print(batch_y)
        batch_x_final = np.zeros((0, 96, 1366, 1))
        for filename in batch_x:
          full_path = '/content/'+str(filename)+'.npy'
          this_array = np.load(full_path).reshape(1, 96, 1366, 1)
          batch_x_final = np.concatenate((batch_x_final, this_array), axis=0)
          
        return np.array(batch_x_final), np.array(batch_y)
        
        #return np.array([
         #   resize(imread('/content/'+file_name+'.npy'), (1366, 96, 1))
          #    for file_name in batch_x]), np.array(batch_y)

In [0]:
class roc_callback(Callback):
    def __init__(self, validation_batch_generator, y_validation_numpy):
        self.my_validation_batch_generator = validation_batch_generator
        self.y_val_numpy = y_validation_numpy

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        prediction_test = self.model.predict_generator(self.my_validation_batch_generator)
        auc_roc_score = roc_auc_score(self.y_val_numpy, prediction_test)
        print('\r roc-auc score on validation data: ', auc_roc_score)
        #print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [0]:
batch_size = 16

In [0]:
my_training_batch_generator = My_Generator(training_file_path, y_train_numpy, batch_size)
my_validation_batch_generator = My_Generator(validation_file_path, y_val_numpy, batch_size)

In [0]:
roc_callbacks = roc_callback(my_validation_batch_generator, y_val_numpy)

In [0]:
model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch = int(18706 // batch_size),
                   epochs = 10,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(1285 // batch_size),
                   callbacks = [roc_callbacks, checkpoint])

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1169/1169 [==============================] - 821s 702ms/step - loss: 0.2194 - acc: 0.9419 - auc: 0.7011 - val_loss: 0.1881 - val_acc: 0.9437 - val_auc: 0.7358
 roc-auc score on validation data:  0.7238592327806718

Epoch 00001: saving model to weights-improvement-3-01-0.94.hdf5
Epoch 2/10
1169/1169 [==============================] - 809s 692ms/step - loss: 0.1972 - acc: 0.9433 - auc: 0.7501 - val_loss: 0.1792 - val_acc: 0.9444 - val_auc: 0.7620
 roc-auc score on validation data:  0.7556011535096218

Epoch 00002: saving model to weights-improvement-3-02-0.94.hdf5
Epoch 3/10
1169/1169 [==============================] - 808s 691ms/step - loss: 0.1859 - acc: 0.9443 - auc: 0.7712 - val_loss: 0.1668 - val_acc: 0.9456 - val_auc: 0.7795
 roc-auc score on validation data:  0.778577555669122

Epoch 00003: saving model to weights-improvement-3-03-0.95.hdf5
Epoch 4/10
1169/1169 [==============================] - 808s 691ms/step - loss: 0.1

In [0]:
model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch = int(18706 // batch_size),
                   epochs = 10,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(1285 // batch_size),
                   callbacks = [roc_callbacks, checkpoint])

Epoch 1/10
1169/1169 [==============================] - 807s 690ms/step - loss: 0.1429 - acc: 0.9495 - auc: 0.8461 - val_loss: 0.1448 - val_acc: 0.9486 - val_auc: 0.8485
 roc-auc score on validation data:  0.8474630912342359

Epoch 00001: saving model to weights-improvement-4-01-0.95.hdf5
Epoch 2/10
1169/1169 [==============================] - 807s 690ms/step - loss: 0.1412 - acc: 0.9497 - auc: 0.8509 - val_loss: 0.1455 - val_acc: 0.9487 - val_auc: 0.8530
 roc-auc score on validation data:  0.8514929330973234

Epoch 00002: saving model to weights-improvement-4-02-0.95.hdf5
Epoch 3/10
1169/1169 [==============================] - 807s 690ms/step - loss: 0.1395 - acc: 0.9502 - auc: 0.8551 - val_loss: 0.1440 - val_acc: 0.9498 - val_auc: 0.8570
 roc-auc score on validation data:  0.8480901358204932

Epoch 00003: saving model to weights-improvement-4-03-0.95.hdf5
Epoch 4/10
1169/1169 [==============================] - 809s 692ms/step - loss: 0.1382 - acc: 0.9505 - auc: 0.8589 - val_loss: 0.1

In [0]:
model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch = int(18706 // batch_size),
                   epochs = 10,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(1285 // batch_size),
                   callbacks = [roc_callbacks, checkpoint])

Epoch 1/10
1169/1169 [==============================] - 816s 698ms/step - loss: 0.1384 - acc: 0.9504 - auc: 0.8403 - val_loss: 0.1991 - val_acc: 0.9470 - val_auc: 0.8419
 roc-auc score on validation data:  0.7756967550626929

Epoch 00001: saving model to weights-improvement-3-01-0.95.hdf5
Epoch 2/10
1169/1169 [==============================] - 817s 699ms/step - loss: 0.1375 - acc: 0.9505 - auc: 0.8434 - val_loss: 0.1812 - val_acc: 0.9423 - val_auc: 0.8449
 roc-auc score on validation data:  0.7685033106470428

Epoch 00002: saving model to weights-improvement-3-02-0.94.hdf5
Epoch 3/10
1169/1169 [==============================] - 817s 699ms/step - loss: 0.1361 - acc: 0.9508 - auc: 0.8464 - val_loss: 0.1941 - val_acc: 0.9446 - val_auc: 0.8478
 roc-auc score on validation data:  0.7833851548473665

Epoch 00003: saving model to weights-improvement-3-03-0.94.hdf5
Epoch 4/10
1169/1169 [==============================] - 818s 700ms/step - loss: 0.1351 - acc: 0.9510 - auc: 0.8492 - val_loss: 0.1

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
!ls

weights-improvement-3-01-0.94.hdf5  weights-improvement-4-01-0.95.hdf5
weights-improvement-3-02-0.94.hdf5  weights-improvement-4-02-0.95.hdf5
weights-improvement-3-03-0.95.hdf5  weights-improvement-4-03-0.95.hdf5
weights-improvement-3-04-0.95.hdf5  weights-improvement-4-04-0.95.hdf5
weights-improvement-3-05-0.95.hdf5  weights-improvement-4-05-0.95.hdf5
weights-improvement-3-06-0.95.hdf5  weights-improvement-4-06-0.95.hdf5
weights-improvement-3-07-0.95.hdf5  weights-improvement-4-07-0.95.hdf5
weights-improvement-3-08-0.95.hdf5  weights-improvement-4-08-0.95.hdf5
weights-improvement-3-09-0.95.hdf5  weights-improvement-4-09-0.95.hdf5
weights-improvement-3-10-0.95.hdf5  weights-improvement-4-10-0.95.hdf5


In [0]:
#os.chdir('/content/new_model_saved')
os.chdir('/content/')
!ls

In [0]:
import shutil

shutil.make_archive('new_model_saved', 'zip', 'new_model_saved')

'/content/new_model_saved.zip'

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'new_model_saved.zip'})
uploaded.SetContentFile('new_model_saved.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1UmrfZwJ89pnnix8cRM4WBXAOkOwYyDxB


# Testing on model

In [0]:
test_batch_generator = My_Generator(testing_file_path, y_test_numpy, batch_size)

In [0]:
print(test_prediction.shape)

print(y_test_numpy.shape)

(5329, 50)
(5329, 50)


In [0]:
test_prediction = model.predict_generator(test_batch_generator)

In [0]:
auc_roc_score = roc_auc_score(y_test_numpy, test_prediction)
print(auc_roc_score)

0.8524437226818802


In [0]:
print(np.around(test_prediction[2], decimals = 3)) 

[0.228 0.    0.007 0.046 0.009 0.009 0.003 0.108 0.022 0.311 0.002 0.003
 0.006 0.012 0.015 0.01  0.027 0.167 0.012 0.051 0.01  0.003 0.021 0.143
 0.009 0.03  0.002 0.01  0.019 0.032 0.021 0.    0.322 0.024 0.    0.006
 0.184 0.582 0.326 0.059 0.018 0.367 0.005 0.001 0.017 0.002 0.041 0.016
 0.003 0.   ]


In [0]:
y_test_numpy[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0])

In [0]:
test_validation_batch_generator = My_Generator(validation_file_path, y_val_numpy, batch_size)

In [0]:
prediction_test = model.predict_generator(test_validation_batch_generator)
#auc_roc_score = roc_auc_score(self.y_val_numpy, prediction_test)

In [0]:
print(prediction_test.shape)

(1825, 50)


In [0]:
print(y_val_numpy.shape)

(1825, 50)


In [0]:
auc_roc_score = roc_auc_score(y_val_numpy, prediction_test)
print(auc_roc_score)

0.8501626197816595


In [0]:
print(prediction_test[2]>=0.4)

[False False False False False False False False False False False False
 False False False False False False False False False False  True False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False]


In [0]:
y_val_numpy[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0])